In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [3]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
import time


start = time.time()
# Define your processing function
def process_chunk(chunk):
    # Drop the specified column
    # chunk = chunk.drop(["Unnamed: 15"], axis=1)
    
    # Filter out rows where "INSTRUMENT" is in the specified list
    chunk = chunk[~chunk["INSTRUMENT"].isin(['FUTIDX', 'FUTINT', 'FUTIVX', 'FUTSTK'])]

    # Update sets with unique symbols and instruments
    unique_symbols.update(chunk["SYMBOL"].unique())
    instrument_symbols.update(chunk["INSTRUMENT"].unique())
    
    # Calculate total rows
    total_rows = len(chunk)
    
    return chunk, total_rows

# Define your parallel processing function
def parallel_processing(chunk):
    processed_chunk, chunk_rows = process_chunk(chunk)
    return processed_chunk, chunk_rows

# Read a chunk of your data
chunk_size = 200000
chunks = pd.read_csv('fobhav.csv', chunksize=chunk_size)

# Define sets to store unique symbols and instruments
unique_symbols = set()
instrument_symbols = set()
total_rows = 0

# Perform parallel processing
with Pool(4) as pool:
    results = pool.map(parallel_processing, chunks)
    
# Aggregate results
processed_chunks = [result[0] for result in results]
total_rows = sum(result[1] for result in results)

# Concatenate processed chunks
df = pd.concat(processed_chunks)

# Access unique symbols and instruments
print("Unique Symbols:", unique_symbols)
print("Unique Instruments:", instrument_symbols)
print("Total Rows Processed:", total_rows)
end = time.time()
print("Time took: ", end-start)

/usr/local/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/pool.py:478: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  iterable = list(iterable)
/usr/local/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/pool.py:478: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  iterable = list(iterable)
/usr/local/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/pool.py:478: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  iterable = list(iterable)
/usr/local/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/pool.py:478: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13) ha

EOFError: Ran out of input

In [4]:
chunk_df = pd.read_csv('fobhav.csv', nrows=10000000)

chunk_df = chunk_df.iloc[:, :-1]
chunk_df.head()
df = chunk_df

In [5]:
import pandas as pd

df['EXPIRY_DT'] = pd.to_datetime(df['EXPIRY_DT'], format='%d-%b-%Y', errors='coerce')
df = df.dropna(subset=['EXPIRY_DT'])
unique_symbols = set()
instrument_symbols = set()

start_date = pd.to_datetime('2015-12-31')
end_date = pd.to_datetime('2025-06-26')

df_filtered = df[(df['EXPIRY_DT'] >= start_date) & (df['EXPIRY_DT'] <= end_date)]
df_filtered = df_filtered[~df_filtered["INSTRUMENT"].isin(['FUTIDX', 'FUTINT', 'FUTIVX', 'FUTSTK'])]
df_filtered.dropna()

unique_symbols.update(df_filtered["SYMBOL"].unique())
instrument_symbols.update(df_filtered["INSTRUMENT"].unique())

min_date = df_filtered['EXPIRY_DT'].min()
max_date = df_filtered['EXPIRY_DT'].max()

print("Minimum Date:", min_date)
print("Maximum Date:", max_date)

df_filtered.reset_index()
df_filtered.head()

Minimum Date: 2019-07-25 00:00:00
Maximum Date: 2025-06-26 00:00:00


,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,TIMESTAMP
414,OPTIDX,BANKNIFTY,2020-09-03,18100.0,CE,0.0,0.0,0.0,6523.60,5660.65,0.0,0.00,25.0,0.0,31-AUG-2020
415,OPTIDX,BANKNIFTY,2020-09-03,18200.0,CE,0.0,0.0,0.0,3122.85,5560.70,0.0,0.00,0.0,0.0,31-AUG-2020
416,OPTIDX,BANKNIFTY,2020-09-03,18300.0,CE,6236.6,6236.6,5684.5,5684.50,5460.75,2.0,12.13,50.0,25.0,31-AUG-2020
417,OPTIDX,BANKNIFTY,2020-09-03,18400.0,CE,0.0,0.0,0.0,3472.20,5360.80,0.0,0.00,0.0,0.0,31-AUG-2020
418,OPTIDX,BANKNIFTY,2020-09-03,18500.0,CE,0.0,0.0,0.0,3394.40,5260.80,0.0,0.00,0.0,0.0,31-AUG-2020


In [10]:
import pandas as pd

# Define the path to your dataset
file_path = 'fobhav.csv'

# Initialize an empty list to store filtered chunks
filtered_chunks = []

# Define the chunk size (adjust as needed)
chunk_size = 100000  # Example chunk size

# Define the date format (assuming 'date_column' is the name of your date column)
date_format = '%Y-%m-%d'  # Adjust the format based on your date format

# Read the dataset in chunks and specify the date format
for chunk in pd.read_csv(file_path, chunksize=chunk_size, parse_dates=['TIMESTAMP'], date_format=pd.to_datetime):
    # Filter the chunk to include only rows with dates between 2019-11-25 and 2020-11-25
    filtered_chunk = chunk[(chunk['TIMESTAMP'] >= '2019-11-25') & (chunk['TIMESTAMP'] <= '2020-11-25')]
    
    # Append the filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate the filtered chunks into a single DataFrame
filtered_df = pd.concat(filtered_chunks)

# Display the first few rows of the filtered DataFrame
print(filtered_df.head())


/var/folders/mp/z340trdx7mjffnpsjn7fbgn80000gn/T/ipykernel_7207/3084785295.py:16: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size, parse_dates=['TIMESTAMP'], date_format=pd.to_datetime):
/var/folders/mp/z340trdx7mjffnpsjn7fbgn80000gn/T/ipykernel_7207/3084785295.py:16: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size, parse_dates=['TIMESTAMP'], date_format=pd.to_datetime):
/var/folders/mp/z340trdx7mjffnpsjn7fbgn80000gn/T/ipykernel_7207/3084785295.py:16: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size, parse_dates=['TIMESTAMP'], date_format=pd.to_datetime):
/var/folders/mp/z340trdx7mjffnpsjn7fbgn8

Empty DataFrame
Columns: [INSTRUMENT, SYMBOL, EXPIRY_DT, STRIKE_PR, OPTION_TYP, OPEN, HIGH, LOW, CLOSE, SETTLE_PR, CONTRACTS, VAL_INLAKH, OPEN_INT, CHG_IN_OI, TIMESTAMP, Unnamed: 15]
Index: []


In [ ]:
# chunk_size = 10000 # this is a number of rows we allocate per chunk
# chunks = pd.read_csv("fobhav.csv", chunksize=chunk_size)

# for i, chunk in enumerate(chunks): # iterate over it, but stop at the first occurrentce of the chunk    
# 	break

# chunk.head()

In [ ]:
import time

total_rows = 0
unique_symbols = set()
instrument_symbols = set()

start = time.time()

for chunk_df in chunks:
    chunk_df.drop(["Unnamed: 15"], axis=1)
    chunk_df = chunk_df[~chunk_df["INSTRUMENT"].isin(['FUTIDX', 'FUTINT', 'FUTIVX', 'FUTSTK'])]

    unique_symbols.update(chunk_df["SYMBOL"].unique())
    instrument_symbols.update(chunk_df["INSTRUMENT"].unique())
    total_rows += len(chunk_df)

end = time.time()
print("Time took: ", end-start)

In [ ]:
len(unique_symbols)   

In [ ]:
len(instrument_symbols)

In [ ]:
print("Remaining columns:", chunk_df.columns.tolist())
